In [27]:
import requests
from bs4 import BeautifulSoup
# import psycopg2
import itertools
import pandas as pd
import numpy as np

In [28]:
header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

In [29]:

df = pd.DataFrame()

#sayfa sayısı kadar döndürür (4.sayfaya kadar döndürdü)
for pageNumber in range(1, 4):
    r = requests.get(f"https://www.kariyer.net/is-ilanlari/bilisim-{pageNumber}", headers=header)
    soup = BeautifulSoup(r.content, "lxml")
    ilanlar = soup.find_all("div", attrs={"class":"list-items"})

    for ilan in ilanlar:

        # Sayfadan ilan linkleri oluşturma
        url = ilan.a.get("href")
        link = "https://www.kariyer.net" + url

        #Yeni Link Çağırma
        r = requests.get(link, headers=header)
        soup = BeautifulSoup(r.content, "lxml")

        #Sayfadaki şirket ismi
        title = soup.find("a", attrs={"class": "company-name"}).text.strip()

        #Şirket niteliklerini alma
        labels = soup.find_all("div", attrs={"class": "label"})
        values = soup.find_all("div", attrs={"class": "value"})

        label_list, value_list = [], []

        #nitelikleri döndürür
        for label in labels:
            a = label.text.strip()
            label_list.append(a)

            for value in values:
                b = value.text.replace("başvuru","").replace("application", "").strip()
                value_list.append(b)
        
        zip_dict = dict(zip(label_list, value_list))

        zip_dict["Firma Adı"] = title


        df_dictionary = pd.DataFrame([zip_dict])
        df = pd.concat([df, df_dictionary], ignore_index=True)


### Veri Ön İşleme

In [30]:
df.head(10)

,Çalışma Şekli,Tecrübe,Başvuru Sayısı,Departman,Firma Adı,Pozisyon Seviyesi,Job Type,Position Level,Application Count,Department,Years of Experience
0,Tam Zamanlı,Tecrübeli / Tecrübesiz,134,Satış Geliştirme,Yapıtaşı Bilgi Teknolojileri Müh. Dnş. Tic. Lt...,NaN,NaN,NaN,NaN,NaN,NaN
1,Tam Zamanlı,NaN,340,İthalat/İhracat,Pratik Aday Servisi,Eleman,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Lunes Teknoloji Arge Ticaret Limited Şİrketi,NaN,Full-Time,Staff,133,Research and Development (R&D),NaN
3,Tam Zamanlı,En az 2 yıl tecrübeli,74,Satış ve Pazarlama,Tanpera Teknoloji ve Endüstriyel Ürünler San. ...,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,Industrial Application Software,NaN,Full-Time,Specialist,999+,Research and Development (R&D),NaN
5,Tam Zamanlı,En az 3 yıl tecrübeli,165,Bilgi Teknolojileri / IT,PAYCORE,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,INTERTOUCH İNTERAKTİF HİZMETLER TİCARET A.Ş,NaN,Full-Time,Manager candidate,53,IT,NaN
7,Tam Zamanlı,NaN,806,Mühendislik,İAS BİLGİ İŞLEM DANIŞMANLIK SANAYİ VE TİC.A.Ş.,Uzman,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,ICterra Bilgi ve İletişim Teknolojileri San. v...,NaN,Full-Time,NaN,78,Finance,More than 4 years of experience
9,Tam Zamanlı,En az 4 yıl tecrübeli,281,Finans,ICTERRA BİLGİ VE İLETİŞİM TEKNOLOJİLERİSAN.VE ...,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:


df2 = df.copy()

#Departman birleştirme
combined_Department = np.where(df2["Departman"].isnull(), df2["Department"], df2["Departman"])
df2["Departman"] = combined_Department
df2.drop(columns=["Department"], inplace=True)


# Çalışma Şekli Birleştirme
combined_job = np.where(df2["Çalışma Şekli"].isnull(), df2["Job Type"], df2["Çalışma Şekli"])
df2["Çalışma Şekli"] = combined_job
df2.drop(columns=["Job Type"], inplace=True)
df2

#Pozisyon Seviyesi Birleştirme
position_Level = np.where(df2["Pozisyon Seviyesi"].isnull(), df2["Position Level"], df2["Pozisyon Seviyesi"])
df2["Pozisyon Seviyesi"] = position_Level
df2.drop(columns=["Position Level"], inplace=True)


#Başvuru Sayısı
app_count = np.where(df2["Başvuru Sayısı"].isnull(), df2["Application Count"], df2["Başvuru Sayısı"])
df2["Başvuru Sayısı"] = app_count
df2.drop(columns=["Application Count"], inplace=True)


#Tecrübe 
Experience = np.where(df2["Tecrübe"].isnull(), df2["Years of Experience"], df2["Tecrübe"])
df2["Tecrübe"] = Experience
df2.drop(columns=["Years of Experience"], inplace=True)
df2

df2 = df2[['Firma Adı', 'Departman', 'Çalışma Şekli', 'Pozisyon Seviyesi', 'Tecrübe', "Başvuru Sayısı"]]

df = df2


In [32]:
df.head()

,Firma Adı,Departman,Çalışma Şekli,Pozisyon Seviyesi,Tecrübe,Başvuru Sayısı
0,Yapıtaşı Bilgi Teknolojileri Müh. Dnş. Tic. Lt...,Satış Geliştirme,Tam Zamanlı,NaN,Tecrübeli / Tecrübesiz,134
1,Pratik Aday Servisi,İthalat/İhracat,Tam Zamanlı,Eleman,NaN,340
2,Lunes Teknoloji Arge Ticaret Limited Şİrketi,Research and Development (R&D),Full-Time,Staff,NaN,133
3,Tanpera Teknoloji ve Endüstriyel Ürünler San. ...,Satış ve Pazarlama,Tam Zamanlı,NaN,En az 2 yıl tecrübeli,74
4,Industrial Application Software,Research and Development (R&D),Full-Time,Specialist,NaN,999+


In [35]:
df.shape

(156, 6)

In [36]:
# excel tablosu oluşturur

file_name = 'MarksData.xlsx'

df.to_excel(file_name)